<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/MCP_AAI_RAG_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## MCP

In [1]:
# mcp_poc.py
import time
import os
import google.generativeai as genai

# --- Simulate Data Sources ---
# In a real scenario, these would be actual databases, APIs, etc.
DATA_SOURCES = {
    "weather_api": {
        "get_current_weather": lambda city: f"The current weather in {city} is sunny with 25°C."
    },
    "stock_data": {
        "get_stock_price": lambda symbol: f"The stock price of {symbol} is $150.75."
    },
    "user_profile_db": {
        "get_user_preference": lambda user_id: f"User {user_id} prefers dark mode and email notifications."
    }
}

# --- Simulate MCP Server ---
# The MCP server dynamically fetches and structures context based on the request.
class MCPServer:
    def __init__(self):
        print("MCP Server initialized. Ready to handle dynamic context requests.")

    def _fetch_data(self, source_name, method_name, *args):
        """
        Simulates fetching data from a specific data source.
        """
        if source_name in DATA_SOURCES and method_name in DATA_SOURCES[source_name]:
            print(f"  MCP Server: Accessing '{source_name}' for '{method_name}'...")
            time.sleep(0.1) # Simulate network latency
            return DATA_SOURCES[source_name][method_name](*args)
        else:
            return f"Error: Data source '{source_name}' or method '{method_name}' not found."

    def get_dynamic_context(self, request_type, **kwargs):
        """
        Dynamically generates context based on the request type.
        This is where the 'protocol' aspect comes in, defining how context is built.
        """
        context = []
        if request_type == "weather_query":
            city = kwargs.get("city", "unknown")
            weather_info = self._fetch_data("weather_api", "get_current_weather", city)
            context.append(f"Weather context: {weather_info}")
        elif request_type == "stock_query":
            symbol = kwargs.get("symbol", "unknown")
            stock_info = self._fetch_data("stock_data", "get_stock_price", symbol)
            context.append(f"Stock context: {stock_info}")
        elif request_type == "personalized_greeting":
            user_id = kwargs.get("user_id", "guest")
            user_pref = self._fetch_data("user_profile_db", "get_user_preference", user_id)
            context.append(f"User preference context: {user_pref}")
        else:
            context.append("No specific context requested or type not recognized.")

        print(f"  MCP Server: Dynamic context generated for '{request_type}'.")
        return "\n".join(context)

# 2. Configuration for Agent
class AgentConfig:
    # Using 'gemini-2.5-flash' as explicitly requested by the user.
    # Note: As of current public documentation, 'gemini-2.0-flash' is the standard.
    # If this model name causes an API error, please verify its availability or
    # revert to a publicly documented model like 'gemini-2.0-flash'.
    LLM_MODEL_NAME: str = "gemini-2.5-flash"
    MAX_AGENT_RETRIES: int = 2 # Max attempts for the agent to refine its answer

# 3. Google Colab / Gemini API Imports and Configuration
GOOGLE_API_KEY = None
try:
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GEMINI')
    print("Google Generative AI configured successfully using Colab Secrets.")
except (ImportError, KeyError):
    print("Not running in Google Colab or 'GEMINI' secret not found. Attempting to get 'GEMINI' environment variable.")
    GOOGLE_API_KEY = os.getenv('GEMINI')

# Initialize Gemini API
if GOOGLE_API_KEY:
    genai.configure(api_key=GOOGLE_API_KEY)
    print(f"Gemini API configured with model: {AgentConfig.LLM_MODEL_NAME}")
else:
    print("Warning: GOOGLE_API_KEY not found. LLM calls will not work.")


# --- Simulate LLM Client ---
# The LLM client makes requests to the MCP server for context and calls the real LLM.
class LLMClient:
    def __init__(self, mcp_server):
        self.mcp_server = mcp_server
        self.model = None
        if GOOGLE_API_KEY:
            try:
                self.model = genai.GenerativeModel(AgentConfig.LLM_MODEL_NAME)
                print(f"LLM Client initialized with Gemini model: {AgentConfig.LLM_MODEL_NAME}")
            except Exception as e:
                print(f"Error initializing Gemini model: {e}. LLM calls will be simulated.")
        else:
            print("LLM Client initialized without Gemini API key. LLM calls will be simulated.")

    def query_llm(self, user_query, request_type, **kwargs):
        """
        Makes a request to the MCP server for context and then calls the real LLM
        to generate a response.
        """
        print(f"\nLLM Client: Received user query: '{user_query}'")
        print(f"LLM Client: Requesting dynamic context from MCP Server (type: '{request_type}')...")

        # Get dynamic context from MCP server
        dynamic_context = self.mcp_server.get_dynamic_context(request_type, **kwargs)

        full_prompt = (
            f"User query: '{user_query}'\n"
            f"Context provided by MCP:\n{dynamic_context}\n\n"
            f"Based on the context, please provide a concise answer."
        )
        print("\n--- LLM Processing ---")
        print(f"LLM Full Prompt:\n{full_prompt}")

        if self.model:
            try:
                # Call the real Gemini LLM
                response = self.model.generate_content([full_prompt])
                if response.candidates:
                    # Access the text from the first part of the first candidate
                    generated_text = response.candidates[0].content.parts[0].text
                    return f"Gemini LLM Response: {generated_text}"
                else:
                    return f"Gemini LLM Response: No content generated. {response.prompt_feedback}"
            except Exception as e:
                print(f"Error calling Gemini LLM: {e}. Falling back to simulated response.")
                # Fallback to simple simulated LLM response if API call fails
                simulated_response = "I'm sorry, I cannot provide real-time data at the moment due to an API issue. "
                if "Weather context" in dynamic_context:
                    simulated_response += f"Based on the context, {dynamic_context.split('Weather context: ')[1].strip()}. "
                elif "Stock context" in dynamic_context:
                    simulated_response += f"Based on the context, {dynamic_context.split('Stock context: ')[1].strip()}. "
                elif "User preference context" in dynamic_context:
                    simulated_response += f"Based on the context, {dynamic_context.split('User preference context: ')[1].strip()}. "
                return f"Simulated LLM Response (API Error): {simulated_response}How can I help you further?"
        else:
            print("Warning: Gemini API not configured. Using simulated LLM response.")
            # Simple simulated LLM response based on context if API is not configured
            simulated_response = "I'm sorry, I cannot provide real-time data in this simulation. "
            if "Weather context" in dynamic_context:
                simulated_response += f"Based on the context, {dynamic_context.split('Weather context: ')[1].strip()}. "
            elif "Stock context" in dynamic_context:
                simulated_response += f"Based on the context, {dynamic_context.split('Stock context: ')[1].strip()}. "
            elif "User preference context" in dynamic_context:
                simulated_response += f"Based on the context, {dynamic_context.split('User preference context: ')[1].strip()}. "
            return f"Simulated LLM Response: {simulated_response}How can I help you further?"

# --- POC Demonstration ---
if __name__ == "__main__":
    print("--- Model Context Protocol (MCP) POC Tutorial ---")

    # 1. Initialize MCP Server
    mcp_server = MCPServer()

    # 2. Initialize LLM Client with MCP Server
    llm_client = LLMClient(mcp_server)

    # --- Scenario 1: Weather Query ---
    print("\n--- Scenario 1: User asks for weather ---")
    response_weather = llm_client.query_llm(
        user_query="What's the weather like in New York?",
        request_type="weather_query",
        city="New York"
    )
    print(f"\nFinal Response to User (Scenario 1):\n{response_weather}")

    # --- Scenario 2: Stock Price Query ---
    print("\n--- Scenario 2: User asks for stock price ---")
    response_stock = llm_client.query_llm(
        user_query="What's the current price of GOOG?",
        request_type="stock_query",
        symbol="GOOG"
    )
    print(f"\nFinal Response to User (Scenario 2):\n{response_stock}")

    # --- Scenario 3: Personalized Greeting ---
    print("\n--- Scenario 3: User expects a personalized greeting ---")
    response_greeting = llm_client.query_llm(
        user_query="Hello there!",
        request_type="personalized_greeting",
        user_id="Alice123"
    )
    print(f"\nFinal Response to User (Scenario 3):\n{response_greeting}")

    print("\n--- MCP POC Tutorial End ---")
    print("This POC demonstrates how MCP allows an LLM to dynamically fetch and integrate")
    print("real-time, specific context from various data sources based on the user's request.")
    print("It highlights direct connection, low latency, and dynamic context provision.")
    print("\nNote: For this script to work with the real Gemini LLM, ensure you have set")
    print("your 'GEMINI' API key as an environment variable or in Google Colab secrets.")


Google Generative AI configured successfully using Colab Secrets.
Gemini API configured with model: gemini-2.5-flash
--- Model Context Protocol (MCP) POC Tutorial ---
MCP Server initialized. Ready to handle dynamic context requests.
LLM Client initialized with Gemini model: gemini-2.5-flash

--- Scenario 1: User asks for weather ---

LLM Client: Received user query: 'What's the weather like in New York?'
LLM Client: Requesting dynamic context from MCP Server (type: 'weather_query')...
  MCP Server: Accessing 'weather_api' for 'get_current_weather'...
  MCP Server: Dynamic context generated for 'weather_query'.

--- LLM Processing ---
LLM Full Prompt:
User query: 'What's the weather like in New York?'
Context provided by MCP:
Weather context: The current weather in New York is sunny with 25°C.

Based on the context, please provide a concise answer.

Final Response to User (Scenario 1):
Gemini LLM Response: It's sunny with 25°C.

--- Scenario 2: User asks for stock price ---

LLM Client:

## AGENTIC AI

In [2]:
# agentic_ai_poc.py
import time
import os
import google.generativeai as genai

# 2. Configuration for Agent
class AgentConfig:
    LLM_MODEL_NAME: str = "gemini-2.5-flash"
    MAX_AGENT_RETRIES: int = 2

# 3. Google Colab / Gemini API Imports and Configuration
GOOGLE_API_KEY = None
try:
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GEMINI')
    print("Google Generative AI configured successfully using Colab Secrets.")
except (ImportError, KeyError):
    print("Not running in Google Colab or 'GEMINI' secret not found. Attempting to get 'GEMINI' environment variable.")
    GOOGLE_API_KEY = os.getenv('GEMINI')

# Initialize Gemini API
if GOOGLE_API_KEY:
    genai.configure(api_key=GOOGLE_API_KEY)
    print(f"Gemini API configured with model: {AgentConfig.LLM_MODEL_NAME}")
else:
    print("Warning: GOOGLE_API_KEY not found. LLM calls will not work.")


# --- Simulate Agents ---
# Each agent has a specific capability.
class ResearchAgent:
    def __init__(self, name="ResearchAgent"):
        self.name = name
        self.model = None
        if GOOGLE_API_KEY:
            try:
                self.model = genai.GenerativeModel(AgentConfig.LLM_MODEL_NAME)
                print(f"  {self.name}: Initialized with Gemini model.")
            except Exception as e:
                print(f"  {self.name}: Error initializing Gemini model: {e}. Research will be simulated.")
        else:
            print(f"  {self.name}: Initialized without Gemini API key. Research will be simulated.")

    def conduct_research(self, topic):
        print(f"  {self.name}: Starting research on '{topic}'...")
        if self.model:
            try:
                prompt = f"Conduct research on the following topic and provide key concepts: {topic}"
                response = self.model.generate_content([prompt])
                if response.candidates:
                    return response.candidates[0].content.parts[0].text
                else:
                    return f"Simulated Research (No LLM content): Could not find information on {topic}. {response.prompt_feedback}"
            except Exception as e:
                print(f"  {self.name}: Error calling Gemini LLM for research: {e}. Falling back to simulated research.")
                time.sleep(0.3) # Simulate research time
                if "AI" in topic:
                    return f"Simulated Research: Found key concepts about {topic}: Machine Learning, Neural Networks, Large Language Models."
                elif "space" in topic:
                    return f"Simulated Research: Found key concepts about {topic}: Planets, Galaxies, Black Holes."
                else:
                    return f"Simulated Research: Found general information about {topic}."
        else:
            time.sleep(0.3) # Simulate research time
            if "AI" in topic:
                return f"Simulated Research: Found key concepts about {topic}: Machine Learning, Neural Networks, Large Language Models."
            elif "space" in topic:
                return f"Simulated Research: Found key concepts about {topic}: Planets, Galaxies, Black Holes."
            else:
                return f"Simulated Research: Found general information about {topic}."

class SummarizationAgent:
    def __init__(self, name="SummarizationAgent"):
        self.name = name
        self.model = None
        if GOOGLE_API_KEY:
            try:
                self.model = genai.GenerativeModel(AgentConfig.LLM_MODEL_NAME)
                print(f"  {self.name}: Initialized with Gemini model.")
            except Exception as e:
                print(f"  {self.name}: Error initializing Gemini model: {e}. Summarization will be simulated.")
        else:
            print(f"  {self.name}: Initialized without Gemini API key. Summarization will be simulated.")

    def summarize_text(self, text):
        print(f"  {self.name}: Summarizing text...")
        if self.model:
            try:
                prompt = f"Summarize the following text concisely: {text}"
                response = self.model.generate_content([prompt])
                if response.candidates:
                    return response.candidates[0].content.parts[0].text
                else:
                    return f"Simulated Summary (No LLM content): Could not summarize. {response.prompt_feedback}"
            except Exception as e:
                print(f"  {self.name}: Error calling Gemini LLM for summarization: {e}. Falling back to simulated summarization.")
                time.sleep(0.2) # Simulate summarization time
                if len(text) > 50:
                    return f"Simulated Summary: '{text[:50]}...' (truncated for POC)"
                return f"Simulated Summary: '{text}'"
        else:
            time.sleep(0.2) # Simulate summarization time
            if len(text) > 50:
                return f"Simulated Summary: '{text[:50]}...' (truncated for POC)"
            return f"Simulated Summary: '{text}'"

class PresentationAgent:
    def __init__(self, name="PresentationAgent"):
        self.name = name
        self.model = None
        if GOOGLE_API_KEY:
            try:
                self.model = genai.GenerativeModel(AgentConfig.LLM_MODEL_NAME)
                print(f"  {self.name}: Initialized with Gemini model.")
            except Exception as e:
                print(f"  {self.name}: Error initializing Gemini model: {e}. Presentation will be simulated.")
        else:
            print(f"  {self.name}: Initialized without Gemini API key. Presentation will be simulated.")

    def format_output(self, content):
        print(f"  {self.name}: Formatting output for presentation...")
        if self.model:
            try:
                prompt = f"Format the following content into a concise report: {content}"
                response = self.model.generate_content([prompt])
                if response.candidates:
                    return f"--- Final Report ---\n{response.candidates[0].content.parts[0].text}\n--- End Report ---"
                else:
                    return f"--- Simulated Report (No LLM content) ---\n{content}\n--- End Report --- (No LLM content)"
            except Exception as e:
                print(f"  {self.name}: Error calling Gemini LLM for formatting: {e}. Falling back to simulated formatting.")
                time.sleep(0.1) # Simulate formatting time
                return f"--- Simulated Report ---\n{content}\n--- End Report ---"
        else:
            time.sleep(0.1) # Simulate formatting time
            return f"--- Simulated Report ---\n{content}\n--- End Report ---"

# --- Simulate Orchestrator ---
# The orchestrator coordinates the agents to achieve a complex goal.
class Orchestrator:
    def __init__(self):
        self.research_agent = ResearchAgent()
        self.summarization_agent = SummarizationAgent()
        self.presentation_agent = PresentationAgent()
        print("Orchestrator initialized with Research, Summarization, and Presentation Agents.")

    def execute_complex_task(self, task_description):
        """
        Orchestrates agents to complete a complex task.
        This demonstrates the coordination aspect.
        """
        print(f"\nOrchestrator: Received complex task: '{task_description}'")

        # Step 1: Research (handled by ResearchAgent)
        print("Orchestrator: Delegating research to ResearchAgent.")
        research_result = self.research_agent.conduct_research(task_description)
        print(f"Orchestrator: Research complete. Result: '{research_result}'")

        # Step 2: Summarize (handled by SummarizationAgent)
        print("Orchestrator: Delegating summarization to SummarizationAgent.")
        summarized_result = self.summarization_agent.summarize_text(research_result)
        print(f"Orchestrator: Summarization complete. Result: '{summarized_result}'")

        # Step 3: Format Output (handled by PresentationAgent)
        print("Orchestrator: Delegating formatting to PresentationAgent.")
        final_output = self.presentation_agent.format_output(summarized_result)
        print(f"Orchestrator: Formatting complete.")

        print("Orchestrator: Complex task execution finished.")
        return final_output

# --- POC Demonstration ---
if __name__ == "__main__":
    print("--- Agentic AI POC Tutorial ---")

    # Initialize the Orchestrator
    orchestrator = Orchestrator()

    # --- Scenario 1: Research and Summarize AI ---
    print("\n--- Scenario 1: Task - 'Research and summarize key concepts about AI' ---")
    report_ai = orchestrator.execute_complex_task("AI")
    print(f"\nFinal Output (Scenario 1):\n{report_ai}")

    # --- Scenario 2: Research and Summarize Space ---
    print("\n--- Scenario 2: Task - 'Research and summarize interesting facts about space' ---")
    report_space = orchestrator.execute_complex_task("space")
    print(f"\nFinal Output (Scenario 2):\n{report_space}")

    # --- Simulate Error Propagation (Illustrative) ---
    # In a real system, an agent might fail, and the orchestrator would need
    # error handling or retry mechanisms. Here, we just simulate a "failure" message.
    print("\n--- Scenario 3: Simulating Error Propagation ---")
    class FaultyAgent:
        def do_something(self):
            print("  FaultyAgent: Attempting task...")
            time.sleep(0.1)
            raise ValueError("Simulated agent failure!")

    orchestrator_with_fault = Orchestrator()
    orchestrator_with_fault.faulty_agent = FaultyAgent() # Inject a faulty agent for demonstration

    try:
        print("Orchestrator: Attempting task with potential fault...")
        orchestrator_with_fault.faulty_agent.do_something()
    except ValueError as e:
        print(f"Orchestrator: Caught an error from a sub-agent: {e}")
        print("This illustrates how errors can propagate in Agentic AI if not handled.")

    print("\n--- Agentic AI POC Tutorial End ---")
    print("This POC demonstrates how an Orchestrator coordinates multiple specialized Agents")
    print("to perform complex tasks. It highlights the modularity but also hints at challenges")
    print("like complex coordination and potential error propagation.")
    print("\nNote: For this script to work with the real Gemini LLM, ensure you have set")
    print("your 'GEMINI' API key as an environment variable or in Google Colab secrets.")


Google Generative AI configured successfully using Colab Secrets.
Gemini API configured with model: gemini-2.5-flash
--- Agentic AI POC Tutorial ---
  ResearchAgent: Initialized with Gemini model.
  SummarizationAgent: Initialized with Gemini model.
  PresentationAgent: Initialized with Gemini model.
Orchestrator initialized with Research, Summarization, and Presentation Agents.

--- Scenario 1: Task - 'Research and summarize key concepts about AI' ---

Orchestrator: Received complex task: 'AI'
Orchestrator: Delegating research to ResearchAgent.
  ResearchAgent: Starting research on 'AI'...
Orchestrator: Research complete. Result: 'Artificial Intelligence (AI) is one of the most transformative technologies of our time, rapidly reshaping industries, societies, and our daily lives.

Here's a comprehensive overview of AI, including its key concepts:

---

## Artificial Intelligence (AI): Key Concepts

### 1. Definition

**Artificial Intelligence (AI)** is a broad field of computer science

## RAG

In [5]:
# rag_poc.py
import time
import os
import hashlib # Used for a simple "embedding" simulation
import google.generativeai as genai

# 2. Configuration for Agent
class AgentConfig:
    LLM_MODEL_NAME: str = "gemini-2.5-flash"
    MAX_AGENT_RETRIES: int = 2

# 3. Google Colab / Gemini API Imports and Configuration
GOOGLE_API_KEY = None
try:
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GEMINI')
    print("Google Generative AI configured successfully using Colab Secrets.")
except (ImportError, KeyError):
    print("Not running in Google Colab or 'GEMINI' secret not found. Attempting to get 'GEMINI' environment variable.")
    GOOGLE_API_KEY = os.getenv('GEMINI')

# Initialize Gemini API
if GOOGLE_API_KEY:
    genai.configure(api_key=GOOGLE_API_KEY)
    print(f"Gemini API configured with model: {AgentConfig.LLM_MODEL_NAME}")
else:
    print("Warning: GOOGLE_API_KEY not found. LLM calls will not work.")


# --- Simulate Vector DB / Knowledge Base ---
# In a real RAG system, this would be a vector database with actual embeddings.
# Here, we use a dictionary where keys are "simulated embeddings" (hashes)
# and values are text chunks.
KNOWLEDGE_BASE = {
    "doc1_ai_history": "Artificial intelligence (AI) has a long history, dating back to the 1950s with early work on symbolic AI.",
    "doc2_llm_basics": "Large Language Models (LLMs) are a type of AI model trained on vast amounts of text data to understand and generate human-like language.",
    "doc3_rag_concept": "Retrieval Augmented Generation (RAG) combines the power of LLMs with external knowledge retrieval to provide more accurate and up-to-date information.",
    "doc4_newton_physics": "Isaac Newton formulated the laws of motion and universal gravitation, laying the foundation for classical mechanics.",
    "doc5_einstein_relativity": "Albert Einstein developed the theory of relativity, revolutionizing our understanding of space, time, gravity, and the universe.",
    "doc6_galileo_astronomy": "Galileo Galilei made significant contributions to astronomy, including observations of Jupiter's moons and phases of Venus.",
    "doc7_hinton_deep_learning": "Geoffrey Hinton is a pioneer in deep learning, known for his work on neural networks and backpropagation.",
    "doc8_ai_flight_planning": "AI agents can optimize flight paths considering weather, fuel efficiency, and air traffic control constraints, leading to safer and more economical travel."
}

# --- Simulate Embedding Model ---
# A very simple hash function to represent an "embedding".
# In reality, this would be a complex neural network producing dense vectors.
def simulate_embedding(text):
    """
    Simulates an embedding by creating a simple hash of the text.
    This is NOT a real embedding but serves for POC matching.
    """
    return hashlib.sha256(text.lower().encode()).hexdigest()[:10] # Use first 10 chars for simplicity

# --- Simulate Retriever ---
# Finds the most "relevant" chunk based on a simple keyword match or simulated embedding similarity.
class Retriever:
    def __init__(self, knowledge_base):
        self.knowledge_base = knowledge_base
        # Pre-compute "embeddings" for the knowledge base for simple lookup
        self.indexed_knowledge = {simulate_embedding(v): v for k, v in knowledge_base.items()}
        print("Retriever initialized with knowledge base.")

    def retrieve_chunks(self, query, top_k=1):
        """
        Simulates retrieving relevant chunks based on the query.
        In a real RAG, this would involve vector similarity search.
        Here, we do a simple keyword matching for demonstration.
        """
        print(f"  Retriever: Searching for relevant chunks for query: '{query}'...")
        time.sleep(0.1) # Simulate retrieval time

        relevant_chunks = []
        query_words = query.lower().split()

        # Simple keyword matching for demonstration
        for chunk_id, chunk_text in self.knowledge_base.items():
            if any(word in chunk_text.lower() for word in query_words):
                relevant_chunks.append(chunk_text)
                if len(relevant_chunks) >= top_k:
                    break # Get top_k matching chunks

        if not relevant_chunks:
            # Fallback: if no keyword match, try a direct "embedding" match (very basic)
            query_embedding = simulate_embedding(query)
            if query_embedding in self.indexed_knowledge:
                relevant_chunks.append(self.indexed_knowledge[query_embedding])

        if not relevant_chunks:
            print("  Retriever: No relevant chunks found.")
            return []
        print(f"  Retriever: Found {len(relevant_chunks)} relevant chunk(s).")
        return relevant_chunks

# --- Simulate Generator (LLM) ---
# Combines the retrieved information with the original query to generate a response.
class Generator:
    def __init__(self):
        self.model = None
        if GOOGLE_API_KEY:
            try:
                self.model = genai.GenerativeModel(AgentConfig.LLM_MODEL_NAME)
                print(f"Generator (Gemini LLM) initialized with model: {AgentConfig.LLM_MODEL_NAME}")
            except Exception as e:
                print(f"Generator: Error initializing Gemini model: {e}. LLM generation will be simulated.")
        else:
            print("Generator (Simulated LLM) initialized without Gemini API key. LLM generation will be simulated.")


    def generate_response(self, original_query, retrieved_chunks):
        """
        Generates a response using the real Gemini LLM, augmented with retrieved context.
        """
        print(f"  Generator: Combining query and retrieved chunks to generate response...")

        context_str = "\n".join([f"- {chunk}" for chunk in retrieved_chunks]) if retrieved_chunks else "No specific context retrieved."

        full_prompt = (
            f"Original user query: '{original_query}'\n"
            f"Retrieved relevant information:\n{context_str}\n\n"
            f"Based on the original query and the retrieved information, provide a comprehensive answer."
        )
        print("\n--- LLM Generation Prompt ---")
        print(f"Prompt to LLM:\n{full_prompt}")

        if self.model:
            try:
                response = self.model.generate_content([full_prompt])
                if response.candidates:
                    generated_text = response.candidates[0].content.parts[0].text
                    return f"Gemini LLM Response: {generated_text}"
                else:
                    return f"Gemini LLM Response: No content generated. {response.prompt_feedback}"
            except Exception as e:
                print(f"Error calling Gemini LLM for generation: {e}. Falling back to simulated response.")
                # Fallback to simple simulated LLM response
                if retrieved_chunks:
                    return (
                        f"Simulated LLM Response (API Error): Based on your query '{original_query}' and the retrieved information:\n"
                        f"{context_str}\n\n"
                        f"I can provide a simulated answer. How can I help you further?"
                    )
                else:
                    return f"Simulated LLM Response (API Error): I couldn't find specific information for '{original_query}' in my knowledge base. Can you rephrase or provide more details?"
        else:
            print("Warning: Gemini API not configured. Using simulated LLM response.")
            # Simple simulated LLM response if API is not configured
            if retrieved_chunks:
                return (
                    f"Simulated LLM Response: Based on your query '{original_query}' and the retrieved information:\n"
                    f"{context_str}\n\n"
                    f"I can provide a simulated answer. How can I help you further?"
                )
            else:
                return f"Simulated LLM Response: I couldn't find specific information for '{original_query}' in my knowledge base. Can you rephrase or provide more details?"


print("--- RAG POC Tutorial ---")
print("This POC demonstrates how RAG works by retrieving relevant information from a knowledge base")
print("and then using that information to augment the LLM's generation, leading to more informed responses.")
print("It highlights the use of external knowledge but also hints at challenges like context staleness")
print("if the knowledge base is not updated.")
print("\nNote: For this script to work with the real Gemini LLM, ensure you have set")
print("your 'GEMINI' API key as an environment variable or in Google Colab secrets.")
print('\n')

# --- POC Demonstration ---
if __name__ == "__main__":
    print("--- Retrieval Augmented Generation (RAG) POC Tutorial ---")
    print("\n--- POC Demonstration ---")
    print('\n')

    # 1. Initialize Retriever with Knowledge Base
    retriever = Retriever(KNOWLEDGE_BASE)

    # 2. Initialize Generator (Simulated LLM)
    generator = Generator()

    # --- Scenario 1: Query about RAG ---
    print("\n--- Scenario 1: User asks about RAG ---")
    user_query_1 = "What is RAG?"
    print(f"User Query: '{user_query_1}'")
    retrieved_chunks_1 = retriever.retrieve_chunks(user_query_1)
    final_response_1 = generator.generate_response(user_query_1, retrieved_chunks_1)
    print(f"\nFinal Response to User (Scenario 1):\n{final_response_1}")

    # --- Scenario 2: Query about Newton ---
    print("\n--- Scenario 2: User asks about Newton ---")
    user_query_2 = "Tell me about Isaac Newton."
    print(f"User Query: '{user_query_2}'")
    retrieved_chunks_2 = retriever.retrieve_chunks(user_query_2)
    final_response_2 = generator.generate_response(user_query_2, retrieved_chunks_2)
    print(f"\nFinal Response to User (Scenario 2):\n{final_response_2}")

    # --- Scenario 3: Query about AI for flight planning (from user's context) ---
    print("\n--- Scenario 3: User asks about AI for flight planning ---")
    user_query_3 = "How can AI help with flight planning?"
    print(f"User Query: '{user_query_3}'")
    retrieved_chunks_3 = retriever.retrieve_chunks(user_query_3)
    final_response_3 = generator.generate_response(user_query_3, retrieved_chunks_3)
    print(f"\nFinal Response to User (Scenario 3):\n{final_response_3}")

    # --- Scenario 4: Query with no direct match ---
    print("\n--- Scenario 4: User asks about something not in KB ---")
    user_query_4 = "What is the capital of France?"
    print(f"User Query: '{user_query_4}'")
    retrieved_chunks_4 = retriever.retrieve_chunks(user_query_4)
    final_response_4 = generator.generate_response(user_query_4, retrieved_chunks_4)
    print(f"\nFinal Response to User (Scenario 4):\n{final_response_4}")

    print("\n--- RAG POC Tutorial End ---")


Google Generative AI configured successfully using Colab Secrets.
Gemini API configured with model: gemini-2.5-flash
--- RAG POC Tutorial ---
This POC demonstrates how RAG works by retrieving relevant information from a knowledge base
and then using that information to augment the LLM's generation, leading to more informed responses.
It highlights the use of external knowledge but also hints at challenges like context staleness
if the knowledge base is not updated.

Note: For this script to work with the real Gemini LLM, ensure you have set
your 'GEMINI' API key as an environment variable or in Google Colab secrets.


--- Retrieval Augmented Generation (RAG) POC Tutorial ---

--- POC Demonstration ---


Retriever initialized with knowledge base.
Generator (Gemini LLM) initialized with model: gemini-2.5-flash

--- Scenario 1: User asks about RAG ---
User Query: 'What is RAG?'
  Retriever: Searching for relevant chunks for query: 'What is RAG?'...
  Retriever: Found 1 relevant chunk(s).
